In [9]:
### Проверить и найти дисконтированные сети среди заблокированных в файле /home/roamability/config/gysvc/relay.config

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import time
import math
from tqdm import tqdm
from importlib import reload
from os.path import join, normpath
from bs4 import BeautifulSoup
import os
import roamability as rb
import re
from xml.etree import ElementTree

downloads = 'C:/Users/balob/Downloads'

In [2]:
file = join(downloads,'test.xml')
tree = ElementTree.parse(file)
root = tree.getroot() # root element can be only one

df_test_result = DataFrame(columns=['SPONSOR','MCCMNC'])

for country in root.find('Reject'):
    try:
        plmn = country.find('PLMN').text
        mccmnc = country.find('IMSI').text
    except:
        pass
    else:
        df_test_result = df_test_result.append(DataFrame([[mccmnc, plmn]],
                                         columns=['SPONSOR','MCCMNC']), ignore_index=True)

df_test_result.replace({'42501':1,'26006':2}, inplace=True)
df_test_result.head(3)

,SPONSOR,MCCMNC
0,1,310
1,1,23215
2,1,23207


In [3]:
# Из базы данных OCS выгружаем список TADIG с NETWORK_ID

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT t.tadig_code AS TADIG
,np.MCC + IIF(LEN(MNC)=1,'0' + MNC,MNC) MCCMNC
,r.SPONSOR_ID AS SPONSOR
,r.DATA_RATE,r.IS_DISCOUNTED
,r.START_DATE
FROM ROAMING_PLAN_RULES r
LEFT JOIN RDB_TADIG_CODES t ON r.network_id=t.network_id
LEFT JOIN ROAMING_PLANS p ON r.roaming_plan_id=p.roaming_plan_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES np ON t.network_id = np.network_id
WHERE r.roaming_plan_id IN (267,268)
AND p.deleted IS NULL
AND r.end_date IS NULL
AND r.IS_DISCOUNTED = 1
'''

df_tadig_mccmnc = pd.read_sql_query(sql_srt, cnxn)

###################################################
# Adjustments
# Remove YUGPM
df_tadig_mccmnc.drop(
    df_tadig_mccmnc[(df_tadig_mccmnc.TADIG=='YUGPM')].index, inplace=True)
# c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\PRICES\Partner_TURTK_YUGPM_not_discounted.msg
###################################################
    
cnxn.close()

df_tadig_mccmnc.head()

,TADIG,MCCMNC,SPONSOR,DATA_RATE,IS_DISCOUNTED,START_DATE
0,AIACW,365840,1.0,0.0257,True,2019-01-01
1,ALBAM,27601,2.0,0.0480,True,2019-04-10
2,ALBEM,27603,2.0,0.0468,True,2019-04-10
3,ALBVF,27602,1.0,0.0030,True,2019-06-01
4,ARG01,72207,1.0,0.0180,True,2019-01-01


In [4]:
df = pd.merge(df_test_result, df_tadig_mccmnc, how='left', on=['MCCMNC','SPONSOR'])
df[(df.IS_DISCOUNTED==1) & (df.START_DATE!='2019-08-01')].sort_values(by='TADIG')

,SPONSOR,MCCMNC,TADIG,DATA_RATE,IS_DISCOUNTED,START_DATE
46,1,21201,MCOK8,0.1000,True,2019-03-01
67,2,29701,MNEPM,0.0228,True,2019-04-10
68,2,50210,MYSMT,0.0228,True,2019-04-10
69,2,50216,MYSMT,0.0228,True,2019-04-10
76,2,41006,PAKTP,0.0228,True,2019-04-10
80,2,52005,THADT,0.0228,True,2019-04-10
83,2,22001,YUGMT,0.0228,True,2019-04-10
84,2,22002,YUGMT,0.0228,True,2019-04-10


In [6]:
df = pd.merge(df_test_result, df_tadig_mccmnc, how='left', on=['MCCMNC','SPONSOR'])
df[(df.IS_DISCOUNTED==1) & (df.START_DATE=='2019-08-01')].sort_values(by='TADIG').to_csv(join(downloads,'unblock_gy.csv'),
                                                                                         index=False)